Code From: https://towardsdatascience.com/implementing-alexnet-cnn-architecture-using-tensorflow-2-0-and-keras-2113e090ad98

In [1]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import os
import time
import pickle
from PIL import Image
import numpy as np
import glob
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow_addons as tfa
import math

/Users/chris/anaconda3/lib/python3.7/site-packages/tensorflow_addons/utils/ensure_tf_install.py:44: UserWarning: You are currently using a nightly version of TensorFlow (2.5.0-dev20201126). 
TensorFlow Addons offers no support for the nightly versions of TensorFlow. Some things might work, some other might not. 
If you encounter a bug, do not file an issue on GitHub.
  UserWarning,


In [2]:
train_images = pickle.load(open("/Users/chris/Documents/GitHub/Stat440-module3/ChrisL/train_images.pickle","rb"))
train_labels = pickle.load(open("/Users/chris/Documents/GitHub/Stat440-module3/ChrisL/train_labels.pickle","rb"))
validation_images = pickle.load(open("/Users/chris/Documents/GitHub/Stat440-module3/ChrisL/validation_images.pickle","rb"))
validation_labels = pickle.load(open("/Users/chris/Documents/GitHub/Stat440-module3/ChrisL/validation_labels.pickle","rb"))
test_images = pickle.load(open("/Users/chris/Documents/GitHub/Stat440-module3/ChrisL/test_images.pickle","rb"))

In [3]:
train_labels = train_labels - 1
validation_labels = validation_labels - 1
test_labels = np.zeros(len(test_images),dtype = "uint8").reshape(-1,1)

In [4]:
train_rotated = tfa.image.rotate(train_images, 18 * math.pi / 180, interpolation='BILINEAR')

In [21]:
train_mean = tfa.image.mean_filter2d(train_images, filter_shape=11)

In [22]:
train_mean.shape

TensorShape([1163, 384, 512, 3])

In [23]:
train_images_all = np.vstack((train_images,train_mean))
train_labels_all = np.vstack((train_labels,train_labels))

In [24]:
train_ds = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
train_all_ds = tf.data.Dataset.from_tensor_slices((train_images_all, train_labels_all))
test_ds = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
validation_ds = tf.data.Dataset.from_tensor_slices((validation_images, validation_labels))

In [25]:
def process_images(image, label):
    # Normalize images to have a mean of 0 and standard deviation of 1
    image = tf.image.per_image_standardization(image)
    # Resize images from 32x32 to 277x277
    image = tf.image.resize(image, (227,227))
    return image, label

In [26]:
train_ds_size = tf.data.experimental.cardinality(train_ds).numpy()
train_all_ds_size = tf.data.experimental.cardinality(train_all_ds).numpy()
test_ds_size = tf.data.experimental.cardinality(test_ds).numpy()
validation_ds_size = tf.data.experimental.cardinality(validation_ds).numpy()
print("Training data size:", train_ds_size)
print("Test data size:", test_ds_size)
print("Validation data size:", validation_ds_size)

Training data size: 1163
Test data size: 1264
Validation data size: 100


In [27]:
train_ds = (train_ds
                  .map(process_images)
                  .shuffle(buffer_size=train_ds_size)
                  .batch(batch_size=32))
train_all_ds = (train_all_ds
                  .map(process_images)
                  .shuffle(buffer_size=train_all_ds_size)
                  .batch(batch_size=32))
test_ds = (test_ds
                  .map(process_images)
                  .batch(batch_size=32))
validation_ds = (validation_ds
                  .map(process_images)
                  .shuffle(buffer_size=train_ds_size)
                  .batch(batch_size=32))

In [28]:
model = keras.models.Sequential([
    keras.layers.Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(227,227,3)),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=384, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=256, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(6, activation='softmax')
])

In [29]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.optimizers.SGD(lr=0.001), metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 55, 55, 96)        34944     
_________________________________________________________________
batch_normalization_5 (Batch (None, 55, 55, 96)        384       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 27, 27, 96)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 27, 27, 256)       614656    
_________________________________________________________________
batch_normalization_6 (Batch (None, 27, 27, 256)       1024      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 13, 13, 256)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 13, 13, 384)      

In [30]:
root_logdir = os.path.join(os.curdir, "logs\\fit\\")
def get_run_logdir():
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)
run_logdir = get_run_logdir()
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
# earlystop = EarlyStopping(monitor= 'val_loss', min_delta=0.001, patience=5, verbose=1)
# callbacks = earlystop

In [31]:
model.fit(train_all_ds,
          epochs=20,
          validation_data=validation_ds,
          validation_freq=1,
          callbacks=[tensorboard_cb])

Epoch 1/20
73/73 [==============================] - 62s 791ms/step - loss: 3.4009 - accuracy: 0.2400 - val_loss: 1.9565 - val_accuracy: 0.3100
Epoch 2/20
73/73 [==============================] - 64s 829ms/step - loss: 2.1247 - accuracy: 0.3869 - val_loss: 1.5166 - val_accuracy: 0.3800
Epoch 3/20
73/73 [==============================] - 65s 838ms/step - loss: 1.7165 - accuracy: 0.4592 - val_loss: 0.9974 - val_accuracy: 0.6400
Epoch 4/20
73/73 [==============================] - 66s 837ms/step - loss: 1.4628 - accuracy: 0.5060 - val_loss: 0.9166 - val_accuracy: 0.6400
Epoch 5/20
73/73 [==============================] - 66s 842ms/step - loss: 1.2618 - accuracy: 0.5542 - val_loss: 0.8729 - val_accuracy: 0.6700
Epoch 6/20
73/73 [==============================] - 66s 843ms/step - loss: 1.1474 - accuracy: 0.5816 - val_loss: 0.8713 - val_accuracy: 0.6900
Epoch 7/20
73/73 [==============================] - 66s 851ms/step - loss: 1.0286 - accuracy: 0.6205 - val_loss: 0.9111 - val_accuracy: 0.6700

In [32]:
prediction = model.predict(test_ds)

In [33]:
yPred = np.argmax(prediction, axis=-1)

In [34]:
model.evaluate(validation_ds)

4/4 [==============================] - 1s 154ms/step - loss: 0.7526 - accuracy: 0.7100


[0.7526460886001587, 0.7099999785423279]

In [35]:
pred_df = pd.read_csv("/Users/chris/Documents/GitHub/Stat440-module3/ChrisL/example.csv")

In [36]:
pred_df['Prediction'] = yPred
pred_df['Prediction'] = pred_df['Prediction'] + 1

In [37]:
pred_df.to_csv("/Users/chris/Documents/GitHub/Stat440-module3/ChrisL/prediction.csv",index=False)